In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('vk.csv')
df.head()

,Unnamed: 0,question,answer
0,0,кредитная карта моментум. ею можно расплачива...,"вы можете, использовать кредитую карту visa/..."
1,1,"на дебетную карту пришло сообщение о ""неразреш...",по данному вопросу вы можете обратиться в сл...
2,2,вопрос. почему не приходят смс о пополнении ка...,по данному вопросу вы можете обратиться в сл...
3,3,"есть зарплатная карта сб, пользуюсь сбол. кро...",по данному вопросу вы можете обратиться в сл...
4,4,"здраствуйте.мне пришло смс ,что одобрен предво...",потребительский кредит может быть выдан при ...


In [2]:
import logging
import os
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
from gensim import corpora


In [3]:
documents = df['question'].tolist()

In [4]:
import re
documents = [str(x)
             .replace("тк","")
             .replace("сбербанк","банк")
             .replace("сбер","банк")
             .replace("сбербанка","банк")
             .replace("банка","банк")
             for x in documents]
documents = [re.sub(r'[^а-яА-Я ]+', '',str(x))
             for x in documents]
print(documents[1])

на дебетную карту пришло сообщение о неразрешенной задолженности никакой задолженности быть не может тк карта дебетная на сколько я понял это банковский косяк и с меня банк захочет взять процент как мне вернуть свои деньги


In [5]:
import nltk
from nltk.corpus import brown
stop_words= nltk.corpus.stopwords.words('russian')
stop_word="пожалуйста здоавствуйте"
for i in stop_words:
        stop_word=  stop_word+" "+i
stoplist = stop_word

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in documents]


In [6]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [7]:
texts = [[token for token in text if frequency[token] > 2] for text in texts]

In [8]:
#lemmatize
import time
start = time.time()
import pymystem3
mystem = pymystem3.Mystem()
texts_lemmatized = [[mystem.lemmatize(token)[0] for token in text] for text in texts]
stop = time.time()

In [9]:
print("Время, затраченное lemmatize на обработку слов "+str((stop - start)))

Время, затраченное lemmatize на обработку слов 62.474364042282104


In [10]:
texts = texts_lemmatized
print(texts[0:3])

[['кредитный', 'карта', 'моментум', 'она', 'расплачиваться', 'зарубежный', 'интернет', 'магазин'], ['дебетный', 'карта', 'приходить', 'сообщение', 'задолженность', 'никакой', 'задолженность', 'тк', 'карта', 'дебетный', 'сколько', 'понимать', 'банковский', 'косяк', 'банк', 'захотеть', 'взять', 'процент', 'вернуть', 'свой', 'деньги'], ['вопрос', 'почему', 'приходить', 'смс', 'пополнение', 'карта', 'ради', 'подключный', 'мобильный', 'банк']]


In [11]:
dictionary = corpora.Dictionary(texts)

#dictionary.filter_extremes(no_below=5, no_above=0.05)

dictionary.save(os.path.join(TEMP_FOLDER, 'deerwester.dict'))  # store the dictionary, for future reference
print(dictionary)


Dictionary(6800 unique tokens: ['кредитный', 'карта', 'моментум', 'она', 'расплачиваться']...)


In [12]:
new_doc = "чат бот"
new_vec = dictionary.doc2bow(new_doc.lower().split())
new_vec

[(4628, 1)]

In [13]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'deerwester.mm'), corpus) 

In [14]:
from gensim import corpora, models, similarities
if os.path.isfile(os.path.join(TEMP_FOLDER, 'deerwester.dict')):
    dictionary = corpora.Dictionary.load(os.path.join(TEMP_FOLDER, 'deerwester.dict'))
    corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'deerwester.mm'))
else:
    print("generate data")

In [17]:
print(dictionary[0])
print(dictionary[1])
print(dictionary[2])

кредитный
карта
моментум


In [34]:
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary,num_topics=170, update_every=1, chunksize=10000, passes=6)

In [35]:
lda.print_topics(num_topics=-1, num_words=12)

[(0,
  '0.197*"социальный" + 0.189*"ипотека" + 0.078*"заводить" + 0.076*"оформлять" + 0.060*"касса" + 0.051*"пенсионер" + 0.046*"должный" + 0.046*"верить" + 0.042*"женщина" + 0.027*"изменение" + 0.017*"карта" + 0.015*"выходной"'),
 (1,
  '0.157*"тыс" + 0.102*"скидывать" + 0.084*"вариант" + 0.076*"благодарить" + 0.055*"рубль" + 0.049*"выплачивать" + 0.048*"нужно" + 0.047*"пользоваться" + 0.047*"ждать" + 0.044*"докладывать" + 0.043*"сколько" + 0.035*"время"'),
 (2,
  '0.437*"мошенник" + 0.105*"данные" + 0.090*"раздел" + 0.045*"автомат" + 0.037*"скоро" + 0.033*"покупать" + 0.032*"мошейница" + 0.032*"слетать" + 0.024*"помнить" + 0.021*"быть" + 0.019*"сотовый" + 0.015*"полгода"'),
 (3,
  '0.192*"кредитный" + 0.162*"карта" + 0.095*"пользоваться" + 0.069*"сказать" + 0.061*"нужно" + 0.042*"процент" + 0.040*"мес" + 0.037*"задолженность" + 0.032*"месяц" + 0.028*"подсказывать" + 0.028*"закрывать" + 0.021*"сразу"'),
 (4,
  '0.342*"дата" + 0.078*"глупость" + 0.043*"транзакция" + 0.043*"верно" + 0.0

In [36]:
lda.show_topics(num_topics=-1,formatted=False,num_words=12)

[(0,
  [('социальный', 0.1968750788355127),
   ('ипотека', 0.18883250202022661),
   ('заводить', 0.078331141657324008),
   ('оформлять', 0.075939101383899379),
   ('касса', 0.059857865436371879),
   ('пенсионер', 0.050917626034586355),
   ('должный', 0.045889079915147298),
   ('верить', 0.045860692297147405),
   ('женщина', 0.041928770446838536),
   ('изменение', 0.02695273332757709),
   ('карта', 0.01692717029534959),
   ('выходной', 0.015078658219926678)]),
 (1,
  [('тыс', 0.15657587668726272),
   ('скидывать', 0.1015107586343871),
   ('вариант', 0.084065039128790625),
   ('благодарить', 0.076215627916342649),
   ('рубль', 0.055190524464592883),
   ('выплачивать', 0.048887313998148267),
   ('нужно', 0.048080620306052743),
   ('пользоваться', 0.047394526486481979),
   ('ждать', 0.046970474847416198),
   ('докладывать', 0.044122845816900555),
   ('сколько', 0.042691247016191702),
   ('время', 0.034594103990570349)]),
 (2,
  [('мошенник', 0.43701993945856454),
   ('данные', 0.1054365181

In [37]:
lda.show_topic(143,topn=12)

[('квартира', 0.095118394113182334),
 ('покупать', 0.081446084307910019),
 ('иметься', 0.081062556611001063),
 ('банк', 0.071276711967572043),
 ('хотеть', 0.070361860296358616),
 ('доля', 0.064428206538780355),
 ('рассказывать', 0.055427626655088567),
 ('документ', 0.050845596563666967),
 ('ипотека', 0.040990016458964731),
 ('перекредитование', 0.031637225892360669),
 ('взять', 0.031326817102146527),
 ('дом', 0.018078419774446432)]

In [38]:
index = similarities.MatrixSimilarity(lda[corpus])


In [39]:
lda.save('model.lda') 
index.save('sims.sims')
